#### As part of the CHESS workshop, we will be simulating a space weather event. We model this event on the 2003 Halloween storm, and first obtain the relevant "ground truth" data for that time period

In [1]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import datetime
import xarray

We begin with hourly-averaged ASCII data obtained from the [NASA OMNI database](https://omniweb.gsfc.nasa.gov/form/dx1.html)

In [2]:
data_folder = Path("../raw_data")
header_file = data_folder / "omni_2003_header.txt"
data_file = data_folder / "omni_2003.asc"

In [3]:
# Reader header info into a list
header_info = []
with open(header_file, 'r') as f:
    lines = f.readlines()

header_info = [x.strip().split()[1] for x in lines[4:]]
print(header_info)

FileNotFoundError: [Errno 2] No such file or directory: '../raw_data/omni2_2003_header.txt'

In [ ]:
# Read data into a dataframe and provide header as column names
# Infer column structure by using all of the lines in the file
df = pd.read_fwf(data_file, names=header_info, infer_nrows = 8760, index_col=False, index=False)

In [ ]:
df.head()

In [ ]:
# Introduce a datetime colum, and remove the year, DOY and hour columns
df["datetime"] = pd.to_datetime(df['YEAR'] * 100000 + 100*df['DOY']+df['Hour'], format='%Y%j%H')
new_index = ['datetime'] + df.keys().tolist()[3:-1]
df = df[new_index]

In [ ]:
df.head()

As an example, we're interested in the 2003 Halloween storm period as our baseline

In [ ]:
start_date = datetime.datetime(year=2003, month=10, day=25)
end_date = datetime.datetime(year=2003, month=11, day=8)

halloween_storm = df[df['datetime'].between(start_date, end_date, inclusive=True)]
halloween_storm.reset_index(drop=True, inplace=True)

In [ ]:
halloween_storm.head()

In [ ]:
halloween_storm_plus = halloween_storm.copy()
halloween_storm_minus = halloween_storm.copy()

We now generate two additional dataframes, one enhanced and one diminished version of the storm

In [ ]:
# Physical quantities we scale by a factor of 5
scaling_factor = 5
for key in ['BX', 'BY', 'BZ', 'SW_Plasma_Temperature', 'SW_Proton_Density']:
        halloween_storm_plus[key] = scaling_factor*halloween_storm_plus[key] - np.nanmean(scaling_factor*halloween_storm_plus[key])

halloween_storm_plus['Scalar_B'] = np.sqrt(halloween_storm_plus['BX']**2 + halloween_storm_plus['BY']**2 + halloween_storm_plus['BZ']**2)        

# These are indices, and do not the same ways as physical quantities.
# We arbitrarily scale them by 1.25
scaling_factor = 1.25
for key in ['Kp', 'ap_index', 'f10.7_index', 'AE-index']:
    halloween_storm_plus[key] = scaling_factor*halloween_storm_plus[key]
    



In [ ]:
# Physical quantities we scale by a factor of 5
scaling_factor = .5
for key in ['BX', 'BY', 'BZ', 'SW_Plasma_Temperature', 'SW_Proton_Density']:
        halloween_storm_minus[key] = scaling_factor*halloween_storm_minus[key] - np.nanmean(scaling_factor*halloween_storm_minus[key])

halloween_storm_minus['Scalar_B'] = np.sqrt(halloween_storm_minus['BX']**2 + halloween_storm_minus['BY']**2 + halloween_storm_minus['BZ']**2)        

# These are indices, and do not the same ways as physical quantities.
# We arbitrarily scale them by 1.25
scaling_factor = .75
for key in ['Kp', 'ap_index', 'f10.7_index', 'AE-index']:
    halloween_storm_minus[key] = scaling_factor*halloween_storm_minus[key]

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(halloween_storm['datetime'], halloween_storm['BZ'],label = 'Orig')
plt.plot(halloween_storm_plus['datetime'],halloween_storm_plus['BZ'], label = 'Enhanced')
plt.plot(halloween_storm_minus['datetime'], halloween_storm_minus['BZ'], label = 'Diminished')
plt.xticks(rotation=45)
plt.title("Bz component of solar wind for Halloween storm")
plt.legend()

Create netCDF files so that these datasets can be used

In [ ]:
xr = xarray.Dataset.from_dataframe(halloween_storm)
xr['BX'].attrs={'units':'nT'}
xr['BY'].attrs={'units':'nT'}
xr['BZ'].attrs={'units':'nT'}
xr['Scalar_B'].attrs={'units':'nT'}
xr['SW_Plasma_Temperature'].attrs={'units':'K'}
xr['SW_Proton_Density'].attrs={'units':'cm^-3'}
xr['ap_index'].attrs={'units':'nT'}
xr['AE-index'].attrs={'units':'nT'}


xr.to_netcdf(data_folder / 'halloween_storm.nc')

In [ ]:
xr = xarray.Dataset.from_dataframe(halloween_storm_plus)
xr['BX'].attrs={'units':'nT'}
xr['BY'].attrs={'units':'nT'}
xr['BZ'].attrs={'units':'nT'}
xr['Scalar_B'].attrs={'units':'nT'}
xr['SW_Plasma_Temperature'].attrs={'units':'K'}
xr['SW_Proton_Density'].attrs={'units':'cm^-3'}
xr['ap_index'].attrs={'units':'nT'}
xr['AE-index'].attrs={'units':'nT'}


xr.to_netcdf(data_folder / 'halloween_storm_plus.nc')

In [ ]:
xr = xarray.Dataset.from_dataframe(halloween_storm_minus)
xr['BX'].attrs={'units':'nT'}
xr['BY'].attrs={'units':'nT'}
xr['BZ'].attrs={'units':'nT'}
xr['Scalar_B'].attrs={'units':'nT'}
xr['SW_Plasma_Temperature'].attrs={'units':'K'}
xr['SW_Proton_Density'].attrs={'units':'cm^-3'}
xr['ap_index'].attrs={'units':'nT'}
xr['AE-index'].attrs={'units':'nT'}


xr.to_netcdf(data_folder / 'halloween_storm_minus.nc')